# Dicionários de padronização dos DFs com códigos e nomes de municípios a partir de DTB_BRASIL_MUNICÍPIO

In [ ]:
# Tratando raw_DTB_BRASIL_MUNICIPIO

# Armazenando raw data em dfs de trabalho para tratamento
df_Codigos_IBGE = Raw_DFs['raw_DTB_BRASIL_MUNICIPIO'].copy()

# Slugifying todas as colunas com nomes de unidades territoriais
cols = ['Nome_UF', 'Nome_Mesorregião', 'Nome_Microrregião', 'Nome_Município']
for  col in cols: 
     df_Codigos_IBGE[col] = df_Codigos_IBGE[col].apply(slugify)

# Armazenando DF tratado no dicionário Treated_DFs
DF_Codigos_IBGE = df_Codigos_IBGE.copy()
Treated_DFs['DF_Codigos_IBGE'] = DF_Codigos_IBGE

In [ ]:
# Cria dicionários IBGE

# Cria dicionário código-município e município-código
CodMunicipio_dict = dict(zip(df_Codigos_IBGE['Código Município Completo'].astype(str), df_Codigos_IBGE['Nome_Município']))
MunicipioCod_dict = dict(zip(df_Codigos_IBGE['Nome_Município'], df_Codigos_IBGE['Código Município Completo'].astype(str)))

# Cria dicionário da mesma natureza associando os códigos das UFs (2 primeiros digitos do código municipal) aos nomes da UFs
CodUF_dict = dict(zip(df_Codigos_IBGE['UF'].astype(str), df_Codigos_IBGE['Nome_UF']))
UFCod_dict = dict(zip(df_Codigos_IBGE['Nome_UF'].astype(str), df_Codigos_IBGE['UF']))

# Cria dicionário associando nomes dos municípios padronizados aos nomes de meso e microrregiões
CodMeso_dict  = dict(zip(df_Codigos_IBGE['Código Município Completo'].astype(str), df_Codigos_IBGE['Nome_Mesorregião']))
CodMicro_dict = dict(zip(df_Codigos_IBGE['Código Município Completo'].astype(str), df_Codigos_IBGE['Nome_Microrregião']))

In [ ]:
# Cria função para adicionar nomenclatura aos dfs a partir do padrão IBGE mapeado nos dicts CodMunic_dict e CodUF_dict
''' DOC:
add_IBGE_std(df, col, cod_ibge = True, drop = []):

Adiciona as colunas cod_municipio, nome_municipio, cod_UF e nome_UF a partir de coluna do df com código do município ou nome do município indicada.
A função necessita dos dicionários criados a partir de tabela do IBGE com códigos e nomes dos municípios (CodMunicipio_dict e MunicipioCod_dict) e realiza 
o mapeamento dos valores da coluna indicada como chave.
No caso de scale = 'mun', adiciona nomes e códigos de municípios e UFs à tabela.
No caso de scale = 'UF', adiciona apenas nomes e códigos de UFs à tabela.
No caso de cod_ibge = True, toma o valor dos códigos do município, cria coluna a partir do dicionário CodMunicpio_dict, e cria cod_UF e nome_UF a partir do dict CodUF_dict
No caso de cod_ibge = False, toma o valor dos nomes dos municípios, cria coluna cod_municipio a partir do dicionário MunicipioCod, e cria cod_UF e nome_UF a partir do dict CodUF_dict
O argumento drop indica colunas que se deseja deletar do df durante o processo.

A função demanda como argumentos obrigatórios: 
  - var df: Um data frame a ser processado
  - str col: O nome de uma coluna chave cujos valores serão mapeados para produção de nova coluna cod_municipio ou cod_IBGE
A função recebe como argumentos opcionais:
  - str: scale: 'mun' por padrão. 'UF' aplica a padronização e inserção de códigos ou nomes apenas de UFs. 'mun' aplica a padronização e inserção de nomes e códigos de municípios e UFs.
  - boolean: cod_ibge: True por padrão. True indica que a coluna chave indica contém os códigos de município do IBGE. False indica que a coluna contém nomes dos município.
  - list of str: drop: [] por padrão. As strings indicadas dentro de drop são deletadas ao final do processo.

'''


def add_IBGE_std(df, col, scale = 'mun', cod_ibge = True, drop = []):
    
    x = df

    if scale == 'mun' and cod_ibge == True :

        # Adiciona colunas numeradas como strings a partir dos códigos do IBGE mapeados como keys em CodMunicipio_dict
        x ['!02cod_municipio']  = x[col].astype(str)
        x ['!03nome_municipio'] = x['!02cod_municipio'].map(CodMunicipio_dict)
        x ['!00cod_UF']         = x[col].astype(str).str[:2]
        x ['!01nome_UF']        = x['!00cod_UF'].map(CodUF_dict)

        # Cria uma lista com os nomes das colunas e a reordena lexicograficamente
        new_order = sorted(x.columns.tolist())

        # Reinsere as colunas no df a partir da lista ordenada new_order
        x = x[new_order]

        # Remove a numeração no nome das colunas
        x.rename({
        '!00cod_UF'        : 'cod_UF',
        '!01nome_UF'       : 'nome_UF',
        '!02cod_municipio' : 'cod_municipio',
        '!03nome_municipio': 'nome_municipio'
        }, axis = 1, inplace = True)

    # Adiciona a colunas a partir do nome do município disponível, padronizado com slugify para corresponder aos nomes padronizados como keys em MunicipioCod_dict   
    elif scale == 'mun' and cod_ibge == False  :
        # Preenche NaN com 'None' para proceder com slugify dos nomes de municipios
        x['!03nome_municipio'] = x[col].fillna('None').apply(slugify)
        x['!02cod_municipio']  = x['!03nome_municipio'].map(MunicipioCod_dict)
        x['!00cod_UF']         = x['!02cod_municipio'].astype(str).str[:2]
        x['!01nome_UF']        = x['!00cod_UF'].map(CodUF_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF'        : 'cod_UF',
        '!01nome_UF'       : 'nome_UF',
        '!02cod_municipio' : 'cod_municipio',
        '!03nome_municipio': 'nome_municipio'
        }, axis = 1, inplace = True)

    elif scale == 'UF' and cod_ibge == True:
        
        # Criando as colunas nome_UF a partir dos códigos do IBGE mapeados como keys em CodUF_dict
        x ['!00cod_UF']   = x[col].astype(str)
        x ['!01nome_UF']  = x['!00cod_UF'].map(CodUF_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF' : 'cod_UF',
        '!01nome_UF': 'nome_UF'
        }, axis = 1, inplace = True)

    # Adiciona colunas a partir do nome do estado disponível, padronizado com slugify correspondendo aos nomes padronizados como keys em UFCod_dict
    elif scale == 'UF' and cod_ibge == False: 
        x['!01nome_UF'] = x[col].fillna('None').apply(slugify)
        x['!00cod_UF']  = x['!01nome_UF'].map(UFCod_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF' : 'cod_UF',
        '!01nome_UF': 'nome_UF'
        }, axis = 1, inplace = True)

    elif scale == 'all' and cod_ibge == True:

        # Adiciona colunas numeradas como strings nome_municipio e nome_UF a partir dos códigos do IBGE mapeados como keys em CodMunicipio_dict
        x['!04cod_municipio']      = x[col].astype(str)
        x['!05nome_municipio']     = x['!04cod_municipio'].map(CodMunicipio_dict)
        x['!00cod_UF']             = x[col].astype(str).str[:2]
        x['!01nome_UF']            = x['!00cod_UF'].map(CodUF_dict)
        x['!02nome_mesorregiao']   = x['!04cod_municipio'].map(CodMeso_dict)
        x['!03nome_microrregiao']  = x['!04cod_municipio'].map(CodMicro_dict)
        
        # Cria uma lista com os nomes das colunas e a reordena lexicograficamente
        new_order = sorted(x.columns.tolist())

        # Reinsere as colunas no df a partir da lista ordenada new_order
        x = x[new_order]

        # Remove a numeração no nome das colunas
        x.rename({
        '!00cod_UF'           : 'cod_UF',
        '!01nome_UF'          : 'nome_UF',
        '!04cod_municipio'    : 'cod_municipio',
        '!05nome_municipio'   : 'nome_municipio',
        '!02nome_mesorregiao' : 'nome_mesorregiao',
        '!03nome_microrregiao': 'nome_microrregiao'
        }, axis = 1, inplace = True)

        # Adiciona a colunas a partir do nome do município disponível, padronizado com slugify para corresponder aos nomes padronizados como keys em MunicipioCod_dict
    elif scale == 'all' and cod_ibge == False: 
        x['!05nome_municipio']    = x[col].fillna('None').apply(slugify)
        x['!04cod_municipio']     = x['!05nome_municipio'].map(MunicipioCod_dict)
        x['!00cod_UF']            = x['!04cod_municipio'].astype(str).str[:2]
        x['!01nome_UF']           = x['!00cod_UF'].map(CodUF_dict)
        x['!02nome_mesorregião']  = x['!04cod_municipio'].map(CodMeso_dict)
        x['!03nome_microrregião'] = x['!04cod_municipio'].map(CodMicro_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF'           : 'cod_UF',
        '!01nome_UF'          : 'nome_UF',
        '!04cod_municipio'    : 'cod_municipio',
        '!05nome_municipio'   : 'nome_municipio',
        '!02nome_mesorregião' : 'nome_mesorregiao',
        '!03nome_microrregião': 'nome_microrregiao'
        }, axis = 1, inplace = True)

    # Se o parâmetro drop for declarado como argumento, deleta as colunas listadas do df
    if len(drop) != 0 :
      x.drop(drop, axis = 1, inplace = True)

    return x
